In [103]:
from openai import OpenAI
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os
import json
from pydantic import BaseModel,Field
from langchain.llms import OpenAI
from dotenv import load_dotenv
from langchain_experimental.pal_chain.base import PALChain
from langchain.chat_models import ChatOpenAI, init_chat_model
from langchain_core.messages import HumanMessage, SystemMessage
from tqdm import tqdm
import re


In [93]:
file_path = 'data/GSM8K/test.jsonl'

with open(file_path, "r", encoding="utf-8") as f:
    dataset = [json.loads(line) for line in f]

In [98]:
print(dataset[0]["question"])
print(dataset[0]["answer"])

Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


In [127]:
load_dotenv()
class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str
load_dotenv()
model=init_chat_model('gpt-4o-mini',model_provider='openai')
messages=[
        SystemMessage(content="You are a math expert. For every question, respond ONLY by calling the 'MathReasoning' tool. You must explain the solution step-by-step using 'steps', and include the final numeric result in 'final_answer'.The result should be a number only, with no units."),
    HumanMessage("Eliza's rate per hour for the first 40 hours she works each week is $10. She also receives an overtime pay of 1.2 times her regular hourly rate. If Eliza worked for 45 hours this week, how much are her earnings for this week?")
]
model_with_tools = model.bind_tools([MathReasoning])
ai_msg = model_with_tools.invoke(messages)

In [128]:
tool_call = ai_msg.tool_calls[0] 
tool_args = tool_call['args']     
tool_args['final_answer']

'460'

In [104]:
def extract_ground_truth(answer: str) -> str:
    match = re.search(r"####\s*(\d+)", answer)
    return match.group(1).strip() if match else ""

def compare_answers(predicted: str, actual: str) -> bool:
    return predicted.strip() == actual.strip()

In [131]:
correct = 0
total = len(dataset)
model_with_tools = model.bind_tools([MathReasoning])
for item in tqdm(dataset):
    question = item["question"]
    true_answer = extract_ground_truth(item["answer"])
    messages = [
        SystemMessage(content="""
        You are a math expert. For every question, you must respond by calling the `MathReasoning` tool.
        Use a list of steps with explanations and intermediate outputs, and provide the final numeric result in `final_answer`.
        Do not answer in plain text.
        """),
        HumanMessage(content=question)
    ]
    try:
        ai_msg = model_with_tools.invoke(messages)
        tool_calls = ai_msg.tool_calls
        if tool_calls and isinstance(tool_calls[0], dict):
            predicted_answer = tool_calls[0]["args"]["final_answer"]
        else:
            predicted_answer = ai_msg.tool_calls[0].args["final_answer"] 
        
        if compare_answers(predicted_answer, true_answer):
            correct += 1
    except Exception as e:
        print(f"Error on question: {question[:60]}... => {e}")
        continue
accuracy = correct / total * 100
print(f"Accuracy: {accuracy:.2f}% ({correct}/{total})")


  1%|          | 13/1319 [00:52<1:25:39,  3.94s/it]

Error on question: Carlos is planting a lemon tree. The tree will cost $90 to p... => 'final_answer'


  2%|▏         | 30/1319 [02:03<1:12:54,  3.39s/it]

Error on question: Gloria is shoe shopping when she comes across a pair of boot... => 'final_answer'


  9%|▉         | 119/1319 [07:35<1:10:18,  3.52s/it]

...

Error on question: Jay is making snowballs to prepare for a snowball fight with... => 'final_answer'


 99%|█████████▉| 1307/1319 [4:09:42<00:53,  4.42s/it]  

Error on question: Dr. Hugo Grumpus and his assistant, Igor, were preparing to ... => 'final_answer'


100%|██████████| 1319/1319 [4:10:32<00:00, 11.40s/it]

Accuracy: 87.19% (1150/1319)
